In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import csv
import os
import copy
import itertools

In [2]:
def select_mode(key):
    number = -1
    
    if key == 107:  # k
        mode = 1
    else:
        mode = 0
    return  mode

In [3]:
def logging_csv(number, mode, landmark_list): 
    # if mode == 0:
    #     pass
    if mode == 1 and (0 <= number <= 9):
        csv_path = 'C:/GroupPJ/USTHGroupProject/Quang/Hand_Getsure_Clean/data/new_keypoint.csv'
        
        # Check if the file exists, if not, create it
        if not os.path.exists(csv_path):
            with open(csv_path, 'w', newline="") as f:
                writer = csv.writer(f)
                # The file will be created but no header is written, just ready to append data

        # Append the data to the CSV file
        with open(csv_path, 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow([number, *landmark_list])
    return


In [4]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point


In [5]:
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list


In [6]:
def store_labels_to_new_file(csv_path, label_csv_path):
    """After collecting all the data in 'new_keypoint.csv', extract labels and store them in 'keypoint_classifier_label_new.csv'."""
    # Check if 'new_keypoint.csv' exists
    if os.path.exists(csv_path):
        # Read the data from 'new_keypoint.csv'
        df = pd.read_csv(csv_path, header=None)
        
        # If the dataframe is not empty, process the data
        if not df.empty:
            # Assuming the first column contains the labels and subsequent columns contain landmark data
            labels = df[0].values
            labels = np.unique(labels)

            # Prepare the data to be written to 'keypoint_classifier_label_new.csv'
            label_data = []
            for label in labels:# Receive the label once (no repeated labels)
                label_data.append(label)
            
            # Write to 'keypoint_classifier_label_new.csv'
            label_df = pd.DataFrame(label_data)
            label_df.to_csv(label_csv_path, index=False, header=False)
            # print(f"Labels and landmarks have been stored in '{label_csv_path}'.")

    else:
        print(f"Error: The file '{csv_path}' does not exist.")


In [7]:
# Initialize video capture
cap = cv2.VideoCapture(0)

import warnings



# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Unable to access the camera.")
    exit()

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,  # Whether to process each frame as a static image
    max_num_hands=1,  # Number of hands to detect
    min_detection_confidence=0.7,  # Minimum confidence value for detecting hands
    min_tracking_confidence=0.5  # Minimum confidence value for hand tracking
)




mode = 0
recCount = 0
csv_path = 'C:/GroupPJ/USTHGroupProject/Quang/Hand_Getsure_Clean/data/new_keypoint.csv'

# Process video frameskk9
while True:
    if recCount == 500: 
        recCount = 0
        mode = 0
    
    if mode == 0:
        store_labels_to_new_file(csv_path, 'C:/GroupPJ/USTHGroupProject/Quang/Hand_Getsure_Clean/data/keypoint_classifier_label.csv')

    if cv2.waitKey(10) & 0xFF == ord('k'):
        mode = 1
        
        # Check if the CSV file exists
        if os.path.exists(csv_path) and os.path.getsize(csv_path) == 0:# Check if file exists but does not have column to parse:
            # If the file exists but is empty, delete the file in local storage
            os.remove(csv_path)
        elif os.path.exists(csv_path):
            # If the file exists, read it into the DataFrame
            df = pd.read_csv(csv_path, header=None)
            store_labels_to_new_file(csv_path, 'C:/GroupPJ/USTHGroupProject/Quang/Hand_Getsure_Clean/data/keypoint_classifier_label.csv')
        else:
            # If the file doesn't exist, create an empty DataFrame with appropriate columns
            # Assuming your CSV has a structure with columns like: [label, x1, y1, x2, y2, ..., xn, yn]
            columns = [0] + [i for i in range(21)] + [j for j in range(21)]  # Example for 21 landmarks
            df = pd.DataFrame(columns=columns)
            # Optionally, save the empty DataFrame to the CSV file
            df.to_csv(csv_path, index=False)
        
    if cv2.waitKey(10) & 0xFF == ord('q'): break

    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting.")
        break

    frame = cv2.flip(frame, 1)  # Mirror the frame
    debug_frame = copy.deepcopy(frame)

    # Convert the frame to RGB (MediaPipe requires RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform hand detection
    results = hands.process(frame_rgb)
    warnings.filterwarnings("ignore", category=UserWarning)
    # If hands are detected, draw landmarks on the frame
    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):

            # Landmark calculation
            Landmark_list = calc_landmark_list(debug_frame, hand_landmarks)

            print(Landmark_list)

            # Conversion to relative coordinates / normalized coordinates
            pre_processed_landmark_list = pre_process_landmark(Landmark_list)

            if mode == 1 and recCount < 500:
                # Create a new number for the current entry (e.g., based on the max label in the CSV)
                number = df[0].max() + 1 if not df.empty else 0
                recCount += 1
                cv2.putText(frame, 'hand record activated', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3)
                print(f"Mode: {mode}, Observation: {recCount}, Label: {number}")
                # Write to the dataset file
                logging_csv(number, mode, pre_processed_landmark_list)
                # Store labels in 'keypoint_classifier_label_new.csv'
                store_labels_to_new_file(csv_path, 'C:/GroupPJ/USTHGroupProject/Quang/Hand_Getsure_Clean/data/keypoint_classifier_label.csv')



            # Draw the landmarks
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display instructions
    cv2.putText(frame, 'Press q to exit ', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3)
    cv2.imshow('Frame', frame)

# Release the capture object and close windows
cap.release()
cv2.destroyAllWindows()


# FOR DISPLAY COORDINATE

In [11]:
# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Initialize video capture
cap = cv2.VideoCapture(0)

# Customize video feed size
desired_width = 1920  # Set your desired width (e.g., 1280 for HD)
desired_height = 720  # Set your desired height (e.g., 720 for HD)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, desired_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, desired_height)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Unable to access the camera.")
    exit()

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,  # Whether to process each frame as a static image
    max_num_hands=1,  # Number of hands to detect
    min_detection_confidence=0.7,  # Minimum confidence value for detecting hands
    min_tracking_confidence=0.5  # Minimum confidence value for hand tracking
)

# Additional variables
mode = 0
recCount = 0
csv_path = 'C:/GroupPJ/USTHGroupProject/Quang/Hand_Getsure_Clean/data/new_keypoint.csv'

# Function definitions for landmark processing
# Add the functions calc_landmark_list, pre_process_landmark, logging_csv, and store_labels_to_new_file here

# Process video frames
while True:
    if recCount == 500: 
        recCount = 0
        mode = 0

    if mode == 0:
        store_labels_to_new_file(csv_path, 'C:/GroupPJ/USTHGroupProject/Quang/Hand_Getsure_Clean/data/keypoint_classifier_label.csv')

    if cv2.waitKey(10) & 0xFF == ord('k'):
        mode = 1

        # Check if the CSV file exists
        if os.path.exists(csv_path) and os.path.getsize(csv_path) == 0:  # Empty file
            os.remove(csv_path)
        elif os.path.exists(csv_path):
            df = pd.read_csv(csv_path, header=None)
            store_labels_to_new_file(csv_path, 'C:/GroupPJ/USTHGroupProject/Quang/Hand_Getsure_Clean/data/keypoint_classifier_label.csv')
        else:
            columns = [0] + [i for i in range(21)] + [j for j in range(21)]
            df = pd.DataFrame(columns=columns)
            df.to_csv(csv_path, index=False)

    if cv2.waitKey(10) & 0xFF == ord('q'): 
        break

    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting.")
        break

    frame = cv2.flip(frame, 1)  # Mirror the frame
    debug_frame = copy.deepcopy(frame)

    # Convert the frame to RGB (MediaPipe requires RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform hand detection
    results = hands.process(frame_rgb)

    # If hands are detected, draw landmarks on the frame
    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):

            # Landmark calculation
            Landmark_list = calc_landmark_list(debug_frame, hand_landmarks)

            # print(Landmark_list)

            # Conversion to relative coordinates / normalized coordinates
            pre_processed_landmark_list = pre_process_landmark(Landmark_list)
            print(pre_processed_landmark_list)

            if mode == 1 and recCount < 500:
                number = df[0].max() + 1 if not df.empty else 0
                recCount += 1
                cv2.putText(frame, 'hand record activated', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3)
                logging_csv(number, mode, pre_processed_landmark_list)
                store_labels_to_new_file(csv_path, 'C:/GroupPJ/USTHGroupProject/Quang/Hand_Getsure_Clean/data/keypoint_classifier_label.csv')

            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display instructions
    cv2.putText(frame, 'Press q to exit ', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3)
    cv2.imshow('Frame', frame)

# Release the capture object and close windows
cap.release()
cv2.destroyAllWindows()


[0.0, 0.0, -0.3209302325581395, -0.14418604651162792, -0.6046511627906976, -0.4232558139534884, -0.5906976744186047, -0.6697674418604651, -0.3209302325581395, -0.6511627906976745, -0.40930232558139534, -0.8465116279069768, -0.5069767441860465, -1.0, -0.48372093023255813, -0.7395348837209302, -0.4325581395348837, -0.6697674418604651, -0.21395348837209302, -0.8465116279069768, -0.3023255813953488, -0.9395348837209302, -0.2837209302325581, -0.6511627906976745, -0.22790697674418606, -0.6232558139534884, -0.013953488372093023, -0.8046511627906977, -0.07906976744186046, -0.8697674418604651, -0.09767441860465116, -0.6, -0.06046511627906977, -0.5813953488372093, 0.19534883720930232, -0.7302325581395349, 0.11627906976744186, -0.7953488372093023, 0.06511627906976744, -0.5953488372093023, 0.08372093023255814, -0.5488372093023256]
[0.0, 0.0, -0.32967032967032966, -0.1043956043956044, -0.6098901098901099, -0.44505494505494503, -0.5164835164835165, -0.7197802197802198, -0.23076923076923078, -0.72527